In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt

## loading data , cleaning the columns names and removing extra spaces

In [2]:
df = pd.read_csv(r"D:\Data since\Epsilon AI\DSP\Mid_project_1\Online Sales Dataset\online_sales_dataset.csv")
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df.rename(columns={
    'invoiceno': 'invoice_num',
    'stockcode': 'stock_code',
    'description': 'description',
    'quantity': 'quantity',
    'invoicedate': 'invoice_date',
    'unitprice': 'unit_price',
    'customerid': 'customer_id',
    'country': 'country',
    'discount': 'discount',
    'paymentmethod': 'payment_method',
    'shippingcost': 'shipping_cost',
    'category': 'category',
    'saleschannel': 'sales_channel',
    'returnstatus': 'return_status',
    'shipmentprovider': 'shipment_provider',
    'warehouselocation': 'warehouse_location',
    'orderpriority': 'order_priority'
}, inplace=True)
df.isnull().sum()

invoice_num              0
stock_code               0
description              0
quantity                 0
invoice_date             0
unit_price               0
customer_id           4978
country                  0
discount                 0
payment_method           0
shipping_cost         2489
category                 0
sales_channel            0
return_status            0
shipment_provider        0
warehouse_location    3485
order_priority           0
dtype: int64

## Handling missing values in cst ID

In [3]:
df['customer_id'] = df['customer_id'].fillna(-999).astype('Int64')

# Handling null values in the shipping_cost on 3 steps as below

In [4]:
# Step 1: Fill with avg shipping for same item + same warehouse
df['shipping_cost'] = df['shipping_cost'].fillna(
    df.groupby(['stock_code', 'warehouse_location'])['shipping_cost'].transform('mean'))
    
# Step 2: Fallback to avg shipping for same item (if Step 1 fails)
df['shipping_cost'] = df['shipping_cost'].fillna(
    df.groupby('stock_code')['shipping_cost'].transform('mean'))
    
# Step 3: Final fallback to global median (if Steps 1-2 fail)
df['shipping_cost'] = df['shipping_cost'].fillna(df['shipping_cost'].median())

## Handling null values in the warehouse_location on 3 steps as below

In [5]:
# Step 1: Find the most common warehouse for each item
item_warehouse_mode = df.groupby('stock_code')['warehouse_location'].agg(lambda x: x.mode()[0])

# Step 2: Fill missing values with the item's mode warehouse
df['warehouse_location'] = df['warehouse_location'].fillna(
    df['stock_code'].map(item_warehouse_mode)
)

# Step 3: If any NA remains (e.g., new items), fill with global mode
df['warehouse_location'] = df['warehouse_location'].fillna(
    df['warehouse_location'].mode()[0]
)

## Extracting date features:

In [6]:
df['order_year'] = pd.to_datetime(df['invoice_date']).dt.year
df['order_month'] = pd.to_datetime(df['invoice_date']).dt.month
df['order_weekday'] = pd.to_datetime(df['invoice_date']).dt.day_name()
df['order_hour'] = pd.to_datetime(df['invoice_date']).dt.hour
df.drop(columns=['invoice_date'], inplace=True)

### checking for dublicate value

In [7]:
int(df.duplicated().sum())

0

### checking for outlier in the numirce columns numeric columns (quantity, unit_price, shipping_cost, discount)

In [8]:
# Check numeric outliers
num_cols = ['quantity', 'unit_price', 'shipping_cost', 'discount']
df[num_cols].describe(percentiles=[0.01, 0.99])

,quantity,unit_price,shipping_cost,discount
count,49782.000000,49782.000000,49782.000000,49782.000000
mean,22.372343,47.537862,17.495345,0.275748
std,17.917774,33.479510,7.041555,0.230077
min,-50.000000,-99.980000,5.000000,0.000000
1%,-40.000000,-69.002300,5.280000,0.008100
50%,23.000000,48.920000,17.500000,0.260000
99%,49.000000,98.930000,29.720000,1.510806
max,49.000000,100.000000,30.000000,1.999764


### handling negative values for quantity

In [9]:
df.loc[df['quantity'] < 0, 'return_status'] = "Returned"
df['is_return'] = df['return_status'] == "Returned"
df['net_quantity'] = df['quantity'].abs()

### handling negative values for unit_price

In [10]:
df['unit_price'] = np.where((df['unit_price'] < 0) & (df['quantity'] >= 0), 0, df['unit_price'])

### handling negative discount

In [11]:
high_discounts = df[df['discount'] > 1.0][['description', 'discount', 'return_status']]

df['discount'] = df['discount'].clip(upper=1.0)

In [12]:
df['unit_price'] = np.where(
    (df['unit_price'] < 0) & (df['quantity'] < 0),
    df['unit_price'],  # Keep valid refunds
    np.where(
        df['unit_price'] < 0,
        0,  # Set other negatives to 0
        df['unit_price']  # Keep positive prices
    )
)
df['is_refund'] = (df['quantity'] < 0)  # Before converting to positive
df[['quantity', 'unit_price']] = df[['quantity', 'unit_price']].abs()

### Data Overview

In [13]:
df['sales'] = df['quantity'] * df['unit_price']

fig = px.histogram(df, x='sales', nbins=50, title='Sales Distribution')
fig.show()

# Categorical summaries
top_countries = df['country'].value_counts().head(10)
fig = px.bar(
	x=top_countries.index,
	y=top_countries.values,
	labels={'x': 'Country', 'y': 'Count'},
	title='Top 10 Countries',
	color=top_countries.index,
	color_discrete_sequence=px.colors.qualitative.Plotly
)
fig.show()

### Temporal Patterns

In [14]:
fig = px.box(df, y=['unit_price', 'quantity'], title='Price/Quantity Ranges')  
fig.update_yaxes(type='log')  # Log scale for skewed data  
fig.show()  

### orrelation Network

In [15]:
corr = df[['quantity', 'unit_price', 'discount', 'sales']].corr()  
fig = px.imshow(corr, text_auto=True, title='Feature Correlations')  
fig.show()  

In [49]:
%%writefile online_sales_dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px

@st.cache_data
def load_data():
    return pd.read_csv(r"D:\Data since\Epsilon AI\DSP\Mid_project_1\Online Sales Dataset\online_sales_dataset_cleaned.csv")  # Replace with your actual file

df = load_data()

# Sidebar filters
st.sidebar.header("🔍 Filters")
country = st.sidebar.multiselect("Country", df["country"].unique())
category = st.sidebar.multiselect("Category", df["category"].unique())
year = st.sidebar.multiselect("Order Year", sorted(df["order_year"].unique()))
return_status = st.sidebar.multiselect("Return Status", df["return_status"].unique())

filtered_df = df.copy()
if country:
    filtered_df = filtered_df[filtered_df["country"].isin(country)]
if category:
    filtered_df = filtered_df[filtered_df["category"].isin(category)]
if year:
    filtered_df = filtered_df[filtered_df["order_year"].isin(year)]
if return_status:
    filtered_df = filtered_df[filtered_df["return_status"].isin(return_status)]

# Tabs
tab1, tab2, tab3, tab4, tab5 = st.tabs([
    "Overview", "Data Table", "Customer Segmentation", "Product Insights", "Trends"
])

# TAB 1: Overview
with tab1:
    st.header("📊 Sales Overview")
    col1, col2, col3 = st.columns(3)
    col1.metric("Total Sales", f"${filtered_df['sales'].sum():,.2f}")
    col2.metric("Total Orders", f"{filtered_df['invoice_num'].nunique():,}")
    col3.metric("Avg. Order Value", f"${filtered_df['sales'].mean():.2f}")

    st.subheader("Sales by Month")
    month_sales = filtered_df.groupby("order_month")["sales"].sum().reset_index()
    st.plotly_chart(px.bar(month_sales, x="order_month", y="sales", labels={"sales": "Sales ($)"}), use_container_width=True)

# TAB 2: Data Table
with tab2:
    st.header("📁 Filtered Data Table")
    st.dataframe(filtered_df)
    st.download_button("Download CSV", data=filtered_df.to_csv(index=False), file_name="filtered_data.csv", mime="text/csv")

# TAB 3: RFM Segmentation
with tab3:
    st.header("🧠 RFM Customer Segmentation")

    # Combine date parts into a datetime
    df["order_date"] = pd.to_datetime(
        df["order_year"].astype(str) + "-" +
        df["order_month"].astype(str) + "-1"
    )

    max_date = df["order_date"].max()

    # RFM Calculation
    rfm = df.groupby("customer_id").agg({
        "order_date": lambda x: (max_date - x.max()).days,
        "invoice_num": "nunique",
        "sales": "sum"
    }).rename(columns={
        "order_date": "Recency",
        "invoice_num": "Frequency",
        "sales": "Monetary"
    }).reset_index()

    # Score each RFM metric from 1 to 4
    rfm["R_Score"] = pd.qcut(rfm["Recency"], q=4, labels=[4, 3, 2, 1])
    rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), q=4, labels=[1, 2, 3, 4])
    rfm["M_Score"] = pd.qcut(rfm["Monetary"], q=4, labels=[1, 2, 3, 4])

    # Combine into single score
    rfm["RFM_Segment"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str) + rfm["M_Score"].astype(str)
    rfm["RFM_Score"] = rfm[["R_Score", "F_Score", "M_Score"]].astype(int).sum(axis=1)

    # Segment labeling (simplified)
    def segment_map(score):
        if score >= 9:
            return "Champions"
        elif score >= 7:
            return "Loyal"
        elif score >= 5:
            return "Potential"
        else:
            return "At Risk"

    rfm["Segment"] = rfm["RFM_Score"].apply(segment_map)

    st.dataframe(rfm)

    st.subheader("📊 Segment Distribution")
    fig = px.histogram(rfm, x="Segment", color="Segment", title="Customer Segment Counts")
    st.plotly_chart(fig, use_container_width=True)

# TAB 4: Product Insights
with tab4:
    st.header("📦 Product Sales Insights")
    top_products = filtered_df.groupby("description")["sales"].sum().nlargest(10).reset_index()
    fig_products = px.bar(top_products, x="sales", y="description", orientation="h")
    st.plotly_chart(fig_products, use_container_width=True)

    st.subheader("Sales by Category")
    cat_sales = filtered_df.groupby("category")["sales"].sum().reset_index()
    st.plotly_chart(px.pie(cat_sales, values="sales", names="category", title="Sales by Category"))

# TAB 5: Trends
with tab5:
    st.header("📈 Sales Trends")

    st.subheader("Hourly Sales")
    hourly_sales = filtered_df.groupby("order_hour")["sales"].sum().reset_index()
    st.plotly_chart(px.line(hourly_sales, x="order_hour", y="sales", markers=True), use_container_width=True)

    st.subheader("Weekday Sales")
    weekday_sales = filtered_df.groupby("order_weekday")["sales"].sum().reset_index()
    st.plotly_chart(px.bar(weekday_sales, x="order_weekday", y="sales"), use_container_width=True)

    st.subheader("Refund Rate Over Time")
    refund_df = filtered_df.groupby("order_month")["is_refund"].mean().reset_index()
    st.plotly_chart(px.line(refund_df, x="order_month", y="is_refund", markers=True, title="Refund Rate by Month"), use_container_width=True)


Overwriting online_sales_dashboard.py


In [50]:
! streamlit run online_sales_dashboard.py

^C
